In [1]:
#thread;;

#require "core";;
#require "core.syntax";;
#require "stdio";;

open Printf;;
open Stdio;;
open Core;;

In [7]:
let executable = "/home/igandhi/Documents/build/bin/klee"

let klee_stats = "/home/igandhi/Documents/build/bin/klee-stats"

let output_flags =
  [ 
  "search=random-path"; "no-forking"; "track-instruction-time"; 
  "libc=uclibc"; "posix-runtime"; "write-paths"; "output-conditions"
  ]
  
let replay_flags =
  [ 
  "search=random-path"; "no-forking"; "track-instruction-time"; 
  "libc=uclibc"; "posix-runtime"
  ]

let stats_flags = [ "print-all"; "to-csv" ]

let directory = "../klee/examples/llvm-coreutils/bitcodes/"

let bitcode = "echo.bc"

(* number of paths to explore with klee *)
let path_n = 1

val executable : string = "/home/igandhi/Documents/build/bin/klee"


val klee_stats : string = "/home/igandhi/Documents/build/bin/klee-stats"


val output_flags : string list =
  ["search=random-path"; "no-forking"; "track-instruction-time";
   "libc=uclibc"; "posix-runtime"; "write-paths"; "output-conditions"]


val replay_flags : string list =
  ["search=random-path"; "no-forking"; "track-instruction-time";
   "libc=uclibc"; "posix-runtime"]


val stats_flags : string list = ["print-all"; "to-csv"]


val directory : string = "../klee/examples/llvm-coreutils/bitcodes/"


val bitcode : string = "echo.bc"


val path_n : int = 1


In [3]:
exception FileError of string

type stderrOutput = 
  | Print
  | Ignore
  | Keep

let flags_to_str (flags : string list) : string =
  String.concat ~sep:" " @@ List.map flags (fun x -> "--" ^ x) 

(** run runs the given command, and returns each line of output of standard out*)
let run (cmd : string) (suppress : stderrOutput) : string list =
  let cmd = match suppress with
  | Print -> cmd
  | Ignore -> cmd ^ " 2> /dev/null"
  | Keep -> cmd ^ " 2>&1"
  in
  let inp = Unix.open_process_in @@ cmd in
  let r = In_channel.input_lines inp in
  In_channel.close inp;
  r
  
(* Outputs are logged as log1.csv, log2.csv... lowest_unused finds the lowest unused log number*)
let lowest_unused () : int = 
    let filenames = run ("ls logs/") Print in
    let num_to_name n = "log" ^ string_of_int n ^ ".csv" in
    let is_unused n = not @@ List.mem filenames (num_to_name n) (String.equal) in
    match List.find (List.init 100 Fun.id) is_unused with
    | Some unused -> unused
    | None -> raise (FileError "All file numbers used")

let write_to_log (headers : string list) (values : string list) : unit =
  let i = lowest_unused () in
  let oc = Out_channel.create ~append:true @@ "logs/log" ^ string_of_int i ^ ".csv" in
  fprintf oc "%s\n" (Option.value (List.hd headers) ~default:"No headers");
  List.iter values (fprintf oc "%s\n");
  Out_channel.close oc

(** get_csv_stats returns the results of "klee-stats" as a tuple of (header, value) *)
let get_csv_stats (directory : string) (cond : string) : string * string =
  let command =
    klee_stats ^ " " ^ directory ^ "klee-last " ^ flags_to_str stats_flags
  in
  let csv_table = run command Print in
  ((Option.value (List.hd csv_table) ~default:"No header") ^ ", Cond", 
  (Option.value (List.hd @@ List.rev csv_table) ~default:"No rows") ^ cond)
  
let print_str_list = fun l -> List.iter l print_endline

exception FileError of string


type stderrOutput = Print | Ignore | Keep


val flags_to_str : string list -> string = <fun>


val run : string -> stderrOutput -> string list = <fun>


val lowest_unused : unit -> int = <fun>


val write_to_log : string list -> string list -> unit = <fun>


val get_csv_stats : string -> string -> string * string = <fun>


val print_str_list : string list -> unit = <fun>


In [4]:
let clear_subdirs (directory : string) : unit = ( run ("rm -rf " ^ directory ^ "*/") Print : string list) |> ignore

val clear_subdirs : string -> unit = <fun>


In [16]:
(*  Given KLEE output, returns a string of where the output is located *)
let get_output_dir (klee_output : string list) : string = 
  let output_str = String.concat ~sep:"\n" klee_output in
  let r = Str.regexp "KLEE: output directory is \"\\(.*\\)\"" in
  Str.search_forward r output_str 0 |> ignore;
  Str.matched_group 1 output_str
 
(*  Given KLEE output, returns a list of replayposition, condition pairs *)
let get_branches (klee_output : string list) : (int * string) list =
  let output_str = String.concat ~sep:" " klee_output in
  let r = Str.regexp "forking on condition: \\([^,]+\\), line number: [0-9]+, position \\([0-9]+\\)" in
  
  let rec match_regs start =
    try
    let next_start = Str.search_forward r output_str start in
    let c = Str.matched_group 1 output_str in
    let p = Str.matched_group 2 output_str in
    (int_of_string p, c) :: match_regs (next_start + 1)
    with _ -> []
  in
  match_regs 0

val get_output_dir : string list -> string = <fun>


val get_branches : string list -> (int * string) list = <fun>


In [38]:
get_output_dir initial_run;;
get_branches initial_run;

- : string =
"/home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-53"


- : (int * string) list =
[(849, "(Sle 32       (SExt w32 (Read w8 0 arg00)))");
 (851, "(Sle 32       (SExt w32 (Read w8 1 arg00)))");
 (853, "(Sle 32       (SExt w32 (Read w8 2 arg00)))");
 (855, "(Sle 32       (SExt w32 (Read w8 3 arg00)))");
 (857, "(Sle 32       (SExt w32 (Read w8 4 arg00)))");
 (859, "(Sle 32       (SExt w32 (Read w8 5 arg00)))");
 (861, "(Sle 32       (SExt w32 (Read w8 6 arg00)))");
 (863, "(Sle 32       (SExt w32 (Read w8 7 arg00)))");
 (865, "(Sle 32       (SExt w32 (Read w8 8 arg00)))");
 (867, "(Sle 32       (SExt w32 (Read w8 9 arg00)))");
 (12723, "(Eq 45 (Read w8 0 arg00))"); (12725, "(Eq 45 (Read w8 0 arg00))");
 (12726, "(Eq 45 (Read w8 0 arg00))"); (12728, "(Eq 45 (Read w8 0 arg00))");
 (12731, "(Eq 45 (Read w8 0 arg00))");
 (12734, "(Eq false      (Eq 0 (Read w8 0 arg00)))")]


In [17]:
let main () =
    let klee_command =
        String.concat ~sep:" " @@ [executable; flags_to_str output_flags; directory ^ bitcode; " --sym-arg 10"]
    in
    let initial_run = (run klee_command Keep : string list) in
    let output_dir = get_output_dir initial_run in
    let branches = get_branches initial_run in
    print_str_list
    let run_branch (p, c) : string * string =
        let klee_command =
        String.concat ~sep:" "
        @@ [ executable; flags_to_str replay_flags; "--replace-condition=" ^ string_of_int p; 
        "--replay-path=\"" ^ output_dir ^ "test000001.path\""; directory ^ bitcode; " --sym-arg 10" ]
        in
        (run klee_command Print : string list) |> ignore;
        get_csv_stats directory c 
    in
    (* header strings, value strings *)
    let (h_str , v_str) = Caml.List.split @@ List.map branches run_branch in
    print_endline @@ Caml.List.hd h_str;
    print_str_list v_str;
    clear_subdirs directory;
    write_to_log h_str v_str

let () = main ()

val main : unit -> unit = <fun>


Initial run in directory: /home/igandhi/Documents/profiler/../klee/examples/llvm-coreutils/bitcodes/klee-out-2


KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
klee: /home/igandhi/Documents/klee/tools/klee/main.cpp:609: static void KleeHandler::loadPathFile(std::__cxx11::string, std::vector<bool>&): Assertion `0 && "unable to open path file"' failed.
/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys15PrintStackTraceERNS_11raw_ostreamE+0x1f)[0x7f8cf13e54bf]
/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys17RunSignalHandlersEv+0x22)[0x7f8cf13e38e2]
/usr/lib/llvm-8/lib/libLLVM-8.so.1(+0x9728c8)[0x7f8cf13e58c8]
/lib/x86_64-linux-gnu/libc.so.6(+0x3ef20)[0x7f8ceff99f20]
/lib/x86_64-linux-gnu/libc.so.6(gsignal+0xc7)[0x7f8ceff99e97]
/lib/x86_64-linux-gnu/libc.so.6(abort+0x141)[0x7f8ceff9b801]
/lib/x86_64-linux-gnu/libc.so.6(+0x3039a)[0x7f8ceff8b39a]
/lib/x86_64-linux-gnu/libc.so.6(+0x30412)[0x7f8ceff8b412]
/home/igandhi/Documents/build/bin/klee(+0x81c88)[0x5

/lib/x86_64-linux-gnu/libc.so.6(+0x3039a)[0x7f6b2ec0f39a]
/lib/x86_64-linux-gnu/libc.so.6(+0x30412)[0x7f6b2ec0f412]
/home/igandhi/Documents/build/bin/klee(+0x81c88)[0x556e03ff8c88]
/home/igandhi/Documents/build/bin/klee(main+0x1b94)[0x556e03fe71e4]
/lib/x86_64-linux-gnu/libc.so.6(__libc_start_main+0xe7)[0x7f6b2ec00b97]
/home/igandhi/Documents/build/bin/klee(_start+0x2a)[0x556e03ff62aa]
Aborted
KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
klee: /home/igandhi/Documents/klee/tools/klee/main.cpp:609: static void KleeHandler::loadPathFile(std::__cxx11::string, std::vector<bool>&): Assertion `0 && "unable to open path file"' failed.
/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys15PrintStackTraceERNS_11raw_ostreamE+0x1f)[0x7f8847b5a4bf]
/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys17RunSignalHandlersEv+0x22)[0x7f8847b588e2]
/usr/lib/llvm-

/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys17RunSignalHandlersEv+0x22)[0x7f38c9e748e2]
/usr/lib/llvm-8/lib/libLLVM-8.so.1(+0x9728c8)[0x7f38c9e768c8]
/lib/x86_64-linux-gnu/libc.so.6(+0x3ef20)[0x7f38c8a2af20]
/lib/x86_64-linux-gnu/libc.so.6(gsignal+0xc7)[0x7f38c8a2ae97]
/lib/x86_64-linux-gnu/libc.so.6(abort+0x141)[0x7f38c8a2c801]
/lib/x86_64-linux-gnu/libc.so.6(+0x3039a)[0x7f38c8a1c39a]
/lib/x86_64-linux-gnu/libc.so.6(+0x30412)[0x7f38c8a1c412]
/home/igandhi/Documents/build/bin/klee(+0x81c88)[0x55ad588cec88]
/home/igandhi/Documents/build/bin/klee(main+0x1b94)[0x55ad588bd1e4]
/lib/x86_64-linux-gnu/libc.so.6(__libc_start_main+0xe7)[0x7f38c8a0db97]
/home/igandhi/Documents/build/bin/klee(_start+0x2a)[0x55ad588cc2aa]
Aborted
KLEE: NOTE: Using POSIX model: /home/igandhi/Documents/build/Debug+Asserts/lib/libkleeRuntimePOSIX.bca
KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
klee: /home/igandhi/Documents/klee/tools/klee/main.cpp:609: static voi

KLEE: NOTE: Using klee-uclibc : /home/igandhi/Documents/build/Debug+Asserts/lib/klee-uclibc.bca
klee: /home/igandhi/Documents/klee/tools/klee/main.cpp:609: static void KleeHandler::loadPathFile(std::__cxx11::string, std::vector<bool>&): Assertion `0 && "unable to open path file"' failed.
/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys15PrintStackTraceERNS_11raw_ostreamE+0x1f)[0x7f27146da4bf]
/usr/lib/llvm-8/lib/libLLVM-8.so.1(_ZN4llvm3sys17RunSignalHandlersEv+0x22)[0x7f27146d88e2]
/usr/lib/llvm-8/lib/libLLVM-8.so.1(+0x9728c8)[0x7f27146da8c8]
/lib/x86_64-linux-gnu/libc.so.6(+0x3ef20)[0x7f271328ef20]
/lib/x86_64-linux-gnu/libc.so.6(gsignal+0xc7)[0x7f271328ee97]
/lib/x86_64-linux-gnu/libc.so.6(abort+0x141)[0x7f2713290801]
/lib/x86_64-linux-gnu/libc.so.6(+0x3039a)[0x7f271328039a]
/lib/x86_64-linux-gnu/libc.so.6(+0x30412)[0x7f2713280412]
/home/igandhi/Documents/build/bin/klee(+0x81c88)[0x55ec4f126c88]
/home/igandhi/Documents/build/bin/klee(main+0x1b94)[0x55ec4f1151e4]
/lib/x86_64-linux-gnu/

Instructions,FullBranches,PartialBranches,NumBranches,UserTime,NumStates,MallocUsage,NumQueries,NumQueryConstructs,NumObjects,WallTime,CoveredInstructions,UncoveredInstructions,QueryTime,SolverTime,CexCacheTime,ForkTime,ResolveTime,QueryCexCacheMisses,QueryCexCacheHits,Breaks,Returns,Switchs,IndirectBrs,Invokes,Resumes,Unreachables,CleanupRets,CatchRets,CatchPads,CatchSwitchs,FNegs,Adds,FAdds,Subs,FSubs,Muls,FMuls,UDivs,SDivs,FDivs,URems,SRems,FRems,Ands,Ors,Xors,Allocas,Loads,Stores,AtomicCmpXchgs,AtomicRMWs,Fences,GetElementPtrs,Truncs,ZExts,SExts,FPTruncs,FPExts,FPToUIs,FPToSIs,UIToFPs,SIToFPs,IntToPtrs,PtrToInts,BitCasts,AddrSpaceCasts,ICmps,FCmps,PHIs,Selects,Calls,Shls,LShrs,AShrs,VAArgs,ExtractElements,InsertElements,ShuffleVectors,ExtractValues,InsertValues,LandingPads,CleanupPads, Cond
171438,58,190,1406,693526.0,0,45929800,21,211,0,570742.0,4067,19329,262668,264556,263295,421,0,21,-1,23585,1074,2,0,0,0,0,0,0,0,0,0,8211,0,1255,0,80,0,6,0,0,1,0,0,131,16,15,5232,51719,34676,0,0,